In [ ]:
import requests
import pandas as pd

In [ ]:
nuclides = ['Ac-225', 'Ag-110m', 'Ba-133', 'Cd-109', 'Ce-139', 'Co-57', 'Co-60', 'Cs-134', 'Ga-67', 'Gd-153', 'Mn-54', 'Ra-223', 'Sn-113', 'Sr-85', 'Tb-161', 'Tl-201', 'Y-88']
path_template = 'https://raw.githubusercontent.com/RomainCoulon/BIPM.RI-II--K1_database/refs/heads/main/{nuclide}_database.json'

data_jsons = []

for nuclide in nuclides:
    path = path_template.format(nuclide=nuclide)
    response = requests.get(path)
    response.raise_for_status()
    data_jsons.append(response.json())

In [ ]:
for data_json in data_jsons:
    nuclide = list(data_json.keys())[1]
    d = data_json[nuclide]
    key_comparisons = [k for k in d.keys() if k.startswith('Key comparison')]

    for kc in key_comparisons:
        if d[kc]['Key Comparison Reference Value (KCRV)'] == 'not evaluated' or 'Degrees of Equivalence' not in d[kc]:
            continue
        doe_dict = d[kc]['Degrees of Equivalence']
        if len(doe_dict) < 3:
            continue
        
        dataset_name = kc.split('.')[-1]
        if dataset_name == 'Co-60(2003_1)' or dataset_name == 'Co-60(2003_2)':
            continue
        print(dataset_name)
        # print(doe_dict)
        df = pd.DataFrame(doe_dict).T
        df.columns = ['value', 'uncertainty']
        df['uncertainty'] = df['uncertainty']/2 # convert 95% to sd
        df.index.name = 'lab'
        # sort by index
        df = df.sort_index()
        df.to_csv(f'{dataset_name}.csv', index=True)